# 微分可能LUTモデルによるMNIST学習

Stochasticモデルに BatchNormalization や Binarize(backward時はHard-Tanh)を加えることで、より一般的なデータに対してLUT回路学習を行います。 

## 事前準備

In [1]:
import os
import shutil
import numpy as np
from tqdm.notebook import tqdm

import torch
import torchvision
import torchvision.transforms as transforms

import binarybrain as bb

異なる閾値で2値化した画像でフレーム数を水増ししながら学習させます。この水増しをバイナリ変調と呼んでいます。

ここではフレーム方向の水増し量を frame_modulation_size で指定しています。

In [2]:
# configuration
data_path             = './data/'
net_name              = 'MnistDifferentiableLutSimple'
data_path             = os.path.join('./data/', net_name)
rtl_sim_path          = '../../verilog/mnist'
rtl_module_name       = 'MnistLutSimple'
output_velilog_file   = os.path.join(data_path, net_name + '.v')
sim_velilog_file      = os.path.join(rtl_sim_path, rtl_module_name + '.v')

epochs                = 4
mini_batch_size       = 64
frame_modulation_size = 15

データセットは PyTorch の torchvision を使います。ミニバッチのサイズも DataLoader で指定しています。
BinaryBrainではミニバッチをフレーム数として FrameBufferオブジェクトで扱います。
バイナリ変調で計算中にフレーム数が変わるためデータセットの準備観点でのミニバッチと呼び分けています。

In [3]:
# dataset
dataset_path = './data/'
dataset_train = torchvision.datasets.MNIST(root=dataset_path, train=True, transform=transforms.ToTensor(), download=True)
dataset_test  = torchvision.datasets.MNIST(root=dataset_path, train=False, transform=transforms.ToTensor(), download=True)
loader_train = torch.utils.data.DataLoader(dataset=dataset_train, batch_size=mini_batch_size, shuffle=True, num_workers=2)
loader_test  = torch.utils.data.DataLoader(dataset=dataset_test,  batch_size=mini_batch_size, shuffle=False, num_workers=2)

## ネットワークの構築

DifferentiableLut に特に何もオプションをつけなければOKです。<br>
バイナリ変調を施すためにネットの前後に RealToBinary層とBinaryToReal層を入れています。<br>
send_command で "binary true" を送ることで、DifferentiableLut の内部の重み係数が 0.0-1.0 の間に拘束されます。

接続数がLUTの物理構成に合わせて、1ノード当たり6個なので層間で6倍以上ノード数が違うと接続されないノードが発生するので、注意してネットワーク設計が必要です。
最終段は各クラス7個の結果を出して Reduce で足し合わせています。こうすることで若干の改善がみられるとともに、加算結果が INT3 相当になるために若干尤度を数値的に見ることができるようです。

In [4]:
# define network
net = bb.Sequential([
            bb.RealToBinary(frame_modulation_size=frame_modulation_size),
            bb.DifferentiableLut([6*6*64]),
            bb.DifferentiableLut([6*64]),
            bb.DifferentiableLut([64]),
            bb.DifferentiableLut([6*6*10]),
            bb.DifferentiableLut([6*10]),
            bb.DifferentiableLut([10]),
            bb.BinaryToReal(frame_integration_size=frame_modulation_size)
        ])
net.set_input_shape([1, 28, 28])

net.send_command("binary true")

loss      = bb.LossSoftmaxCrossEntropy()
metrics   = bb.MetricsCategoricalAccuracy()
optimizer = bb.OptimizerAdam()

optimizer.set_variables(net.get_parameters(), net.get_gradients())

## 学習の実施

load_networks/save_networks で途中結果を保存/復帰可能できます。ネットワークの構造が変わると正常に読み込めなくなるので注意ください。
(その場合は新しいネットをsave_networksするまで一度load_networks をコメントアウトください)

tqdm などを使うと学習過程のプログレス表示ができて便利です。

In [5]:
#bb.load_networks(data_path, net)

# learning
for epoch in range(epochs):
    # learning
    loss.clear()
    metrics.clear()
    with tqdm(loader_train) as t:
        for images, labels in t:
            x_buf = bb.FrameBuffer.from_numpy(np.array(images).astype(np.float32))
            t_buf = bb.FrameBuffer.from_numpy(np.identity(10)[np.array(labels)].astype(np.float32))

            y_buf = net.forward(x_buf, train=True)
            
            dy_buf = loss.calculate(y_buf, t_buf)
            metrics.calculate(y_buf, t_buf)
            
            net.backward(dy_buf)

            optimizer.update()
            
            t.set_postfix(loss=loss.get(), acc=metrics.get())

    # test
    loss.clear()
    metrics.clear()
    for images, labels in loader_test:
        x_buf = bb.FrameBuffer.from_numpy(np.array(images).astype(np.float32))
        t_buf = bb.FrameBuffer.from_numpy(np.identity(10)[np.array(labels)].astype(np.float32))

        y_buf = net.forward(x_buf, train=False)

        loss.calculate(y_buf, t_buf)
        metrics.calculate(y_buf, t_buf)

    print('epoch[%d] : loss=%f accuracy=%f' % (epoch, loss.get(), metrics.get()))

    bb.save_networks(data_path, net)

  0%|          | 0/938 [00:00<?, ?it/s]

epoch[0] : loss=1.640124 accuracy=0.893237


  0%|          | 0/938 [00:00<?, ?it/s]

epoch[1] : loss=1.630475 accuracy=0.886142


  0%|          | 0/938 [00:00<?, ?it/s]

epoch[2] : loss=1.615871 accuracy=0.899224


  0%|          | 0/938 [00:00<?, ?it/s]

epoch[3] : loss=1.616489 accuracy=0.892905


## FPGA用Verilog出力

最後に学習したネットワークを Verilog 出力します。
MNISTのサイズである 28x28=784bit の入力を 10bit の分類をして出力するだけのシンプルなモジュールを出力します。

In [6]:
# export verilog
bb.load_networks(data_path, net)

# 結果を出力
with open(output_velilog_file, 'w') as f:
    f.write('`timescale 1ns / 1ps\n\n')
    bb.dump_verilog_lut_layers(f, module_name=rtl_module_name, net=net)

# Simulation用ファイルに上書きコピー
shutil.copyfile(output_velilog_file, sim_velilog_file)

'../../verilog/mnist\\MnistLutSimple.v'

In [7]:
# シミュレーション用データファイル作成
with open(os.path.join(rtl_sim_path, 'mnist_test.txt'), 'w') as f:
    bb.dump_verilog_readmemb_image_classification(f ,loader_test)

## モデルの内部の値を取得する

Verilog以外の言語やFPGA以外に適用したい場合、接続とLUTテーブルの2つが取得できれば同じ計算をするモデルをインプリメントすることが可能です。

### 事前準備
そのままだと勾配はリセットされているので少しだけ逆伝搬を実施します

In [8]:
# 最新の保存データ読み込み
bb.load_networks(data_path, net)

# layer を取り出す
layer0 = net[1]
layer1 = net[2]
layer2 = net[3]

### 接続を取得する

LUTモデルは get_connection_list() にて接続行列を取得できます。<br>
ここでの各出力ノードは、6つの入力と接続されており、layer0 の出力ノードは 1024 個あるので、1024x6 の行列が取得できます。

In [9]:
connection_mat = np.array(layer0.get_connection_list())
print(connection_mat.shape)
connection_mat

(2304, 6)


array([[285, 227, 432, 459, 774, 502],
       [246,   4, 148, 633, 287, 638],
       [590, 679, 175, 507, 123, 196],
       ...,
       [191, 275,  73, 770, 129, 143],
       [188, 251, 686, 736, 607, 751],
       [414, 130, 670, 459, 210, 578]])

### FPGA化する場合のLUTテーブルを取得する

LUT化する場合のテーブルを取得します。<br>
6入力のLUTモデルなので $ 2^6 = 64 $ 個のテーブルがあります。<br>
モデル内に BatchNormalization 等を含む場合はそれらも加味して最終的にバイナリLUTにする場合に適した値を出力します。

In [10]:
lut_mat = np.array(layer0.get_lut_table_list())
print(lut_mat.shape)
lut_mat

(2304, 64)


array([[False,  True, False, ...,  True,  True,  True],
       [ True,  True,  True, ..., False, False, False],
       [False,  True, False, ...,  True,  True,  True],
       ...,
       [False,  True, False, ...,  True, False, False],
       [ True, False, False, ..., False, False, False],
       [False, False,  True, ...,  True, False, False]])

### 重み行列を覗いてみる

6入力のLUTモデルなので $ 2^6 = 64 $ 個のテーブルがあります。<br>
W() にて bb.Tensor 型で取得可能で、numpy() にて ndarray に変換できます。

In [11]:
W = layer0.W().numpy()
print(W.shape)
W

(2304, 64)


array([[0.46053913, 0.48969   , 0.43049288, ..., 0.55340844, 0.54644394,
        0.54634225],
       [0.5031644 , 0.49801937, 0.4990038 , ..., 0.47459334, 0.44045565,
        0.46549785],
       [0.4600248 , 0.48103702, 0.4552786 , ..., 0.5398099 , 0.5427659 ,
        0.55870146],
       ...,
       [0.4812582 , 0.52008146, 0.45319328, ..., 0.54473335, 0.51105785,
        0.49067914],
       [0.47382542, 0.45093644, 0.4833005 , ..., 0.4593086 , 0.43164894,
        0.44412497],
       [0.49397233, 0.49554572, 0.51608425, ..., 0.54779464, 0.50517625,
        0.50927866]], dtype=float32)

### 勾配を覗いてみる

同様に dW() でW の勾配が取得できます

In [12]:
# そのままだとすべて0なので、1回だけbackward実施
for images, labels in loader_test:
    x_buf = bb.FrameBuffer.from_numpy(np.array(images).astype(np.float32))
    t_buf = bb.FrameBuffer.from_numpy(np.identity(10)[np.array(labels)].astype(np.float32))
    y_buf = net.forward(x_buf, train=True)
    net.backward(loss.calculate(y_buf, t_buf))
    break

dW = layer0.dW().numpy()
print(dW.shape)
dW

(2304, 64)


array([[-6.1715138e-05, -5.8559817e-07, -1.5615504e-05, ...,
        -8.0421364e-07, -9.7475481e-07, -2.1820463e-07],
       [-3.7336437e-04, -4.7944486e-05, -1.2445479e-04, ...,
         4.6901969e-06, -3.0441993e-06,  1.5633991e-06],
       [ 4.2195839e-05,  1.4065794e-05, -1.6487029e-05, ...,
         3.6926622e-08, -6.8869122e-07, -2.2956237e-07],
       ...,
       [ 1.3722657e-04,  3.6448604e-05, -1.9422005e-06, ...,
        -8.5242846e-07, -1.9303056e-06, -8.5227316e-07],
       [-7.0320442e-04,  4.1594882e-03, -2.3440144e-04, ...,
        -2.9756280e-05, -8.3830491e-05, -9.9187655e-06],
       [-7.9892622e-04, -3.3449021e-04, -3.1547446e-04, ...,
         2.8128541e-04,  3.5469886e-04,  1.1414266e-04]], dtype=float32)